In [ ]:
import json
from pathlib import Path
from tqdm import tqdm
import math
import pandas as pd

# Path to your JSONL file
file_path = "/project/flame/asetlur/rc_qw4bins_ansbench_n4_t16_l16k2k.jsonl"

# Each line is a JSON list, merge all into one list, then make a DataFrame from the merged list of dicts
import json

all_items = []
with open(file_path, "r") as f:
    for line in f:
        items = json.loads(line)
        for item in items:
            # If 'metadata' exists and is a dictionary, merge it into the item
            metadata = item.pop("metadata", {})
            if isinstance(metadata, dict):
                # Avoid overwriting existing keys in item when merging metadata
                for k, v in metadata.items():
                    if k not in item:
                        item[k] = v
                    else:
                        # If key collision, optionally rename metadata keys, or overwrite
                        item[f"metadata_{k}"] = v
            all_items.append(item)

df = pd.DataFrame(all_items)

print(f"Loaded {len(df)} rows")
print(f"Dataframe shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
# print(df.head())





Loaded 6400 rows
Dataframe shape: (6400, 20)
Columns: ['text', 'output_text', 'n_predicted', 'reward', 'logprobs', 'ref_logprobs', 'input_ids', 'labels', 'group_id', 'finished', 'prompt_tokens', 'output_tokens', 'visual_features', 'model_version', 'rollout_index', 'cycle_step', 'turn_type', 'turn_number', 'problem_id', 'sample_id']


In [44]:
# Filter for turn_type == "reasoning"
reasoning_df = df[df["turn_type"] == "reasoning"].copy()

# Compute 'turn' as cycle_step // 2
reasoning_df["turn"] = reasoning_df["cycle_step"] // 2

# Group by 'turn' and compute the mean reward
grouped_turns = reasoning_df.groupby("turn")["reward"].mean()

print("Average reward by turn (for turn_type == 'reasoning'):")
print(grouped_turns)


Average reward by turn (for turn_type == 'reasoning'):
turn
0    0.3250
1    0.3425
2    0.3400
3    0.3425
4    0.3500
5    0.3550
6    0.3650
7    0.3675
Name: reward, dtype: float64


In [41]:
df.sample_id[2]

0.0